<a href="https://colab.research.google.com/github/Baco23/github-course/blob/master/LSTM-lot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler


Prepare/Generate data set //
let´s imagine that we have history of lottery by generating 60 numbers


In [0]:
df = pd.DataFrame(np.random.randint(1,60,size=(60, 6)), columns=list('ABCDEF'))

The top games look like this (A B C D E F are balls that take range 1 to 60 each):

In [6]:
df.head()

,A,B,C,D,E,F
0,2,43,4,14,15,44
1,24,1,1,30,4,24
2,22,10,33,58,2,55
3,23,51,25,35,46,39
4,36,30,11,4,41,13


Next we need to normalize data.

In [0]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

Lets define hyper params of or model

In [0]:
number_of_rows = df.values.shape[0] #all our games
window_length = 5 #amount of past games we need to take in consideration for prediction
number_of_features = df.values.shape[1]

Create train dataset and labels for each row. It should have format for keras lstm model(row, window sixe, balls)

In [0]:
train = np.empty([number_of_rows-window_length, window_length, number_of_features], dtype=float)

label = np.empty([number_of_rows-window_length, number_of_features], dtype=float)

window_length = 5

for i in range(0, number_of_rows-window_length):
  train[i]=transformed_df.iloc[i:i+window_length, 0: number_of_features]
  label[i]=transformed_df.iloc[i+window_length: i+window_length+1, 0:number_of_features]

Shapes

In [12]:
train.shape

(55, 5, 6)

In [13]:
label.shape

(55, 6)

In [14]:
train[0]

array([[-1.42253464,  0.99289281, -1.2491958 , -1.17473955, -0.84276502,
         0.59871991],
       [-0.00429769, -1.64644252, -1.41649881, -0.10476804, -1.49029111,
        -0.53450389],
       [-0.13322832, -1.08087066,  0.36806662,  1.7676821 , -1.60802312,
         1.221993  ],
       [-0.068763  ,  1.49562335, -0.07807474,  0.22959805,  0.98208124,
         0.31541396],
       [ 0.76928611,  0.17595569, -0.85882211, -1.84347175,  0.6877512 ,
        -1.15777698]])

In [15]:
train[1]

array([[-0.00429769, -1.64644252, -1.41649881, -0.10476804, -1.49029111,
        -0.53450389],
       [-0.13322832, -1.08087066,  0.36806662,  1.7676821 , -1.60802312,
         1.221993  ],
       [-0.068763  ,  1.49562335, -0.07807474,  0.22959805,  0.98208124,
         0.31541396],
       [ 0.76928611,  0.17595569, -0.85882211, -1.84347175,  0.6877512 ,
        -1.15777698],
       [ 0.64035547, -1.01802934,  1.37188467,  0.83145703, -0.84276502,
        -0.02455318]])

In [16]:
label[0]

array([ 0.64035547, -1.01802934,  1.37188467,  0.83145703, -0.84276502,
       -0.02455318])

In [17]:
label[1]

array([ 1.156078  , -0.45245749,  0.92574331, -0.50600736,  1.04094724,
       -0.70448746])

Our LSTM model

In [0]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import numpy as np

batch_size = 25

Using TensorFlow backend.

In [0]:
model = Sequential()
model.add(LSTM(32,
               input_shape=(window_length, number_of_features),
               return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32,
               return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(number_of_features))
model.compile(loss='mse', optimizer='rmsprop')

Training

In [22]:
model.fit(train, label,
          batch_size=64, epochs=5000)

Epoch 1/5000
55/55 [==============================] - 0s 250us/step - loss: 0.0340
Epoch 2/5000
55/55 [==============================] - 0s 195us/step - loss: 0.0442
Epoch 3/5000
55/55 [==============================] - 0s 142us/step - loss: 0.0401
Epoch 4/5000
55/55 [==============================] - 0s 154us/step - loss: 0.0361
Epoch 5/5000
55/55 [==============================] - 0s 147us/step - loss: 0.0464
Epoch 6/5000
55/55 [==============================] - 0s 150us/step - loss: 0.0552
Epoch 7/5000
55/55 [==============================] - 0s 141us/step - loss: 0.0429
Epoch 8/5000
55/55 [==============================] - 0s 148us/step - loss: 0.0388
Epoch 9/5000
55/55 [==============================] - 0s 165us/step - loss: 0.0491
Epoch 10/5000
55/55 [==============================] - 0s 174us/step - loss: 0.0478
Epoch 11/5000
55/55 [==============================] - 0s 150us/step - loss: 0.0526
Epoch 12/5000
55/55 [==============================] - 0s 138us/step - loss: 0.0497
E

KeyboardInterrupt: ignored